"""
The MIT License (MIT)

Copyright (c) 2017 Chris Scott

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM,
DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE
OR OTHER DEALINGS IN THE SOFTWARE.
"""

<a target="_blank" href="https://colab.research.google.com/github/ABbiodiversity/Recognizer_Ovenbird/blob/main/Ovenbird_Recognizer_Processing.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

#Adjust the Colab link!

### Import Packages

Import packages to run the model.  Updates to packages since the model was originally created may result in warnings when running the model.  As of March 2024 the most up to date versions of the packages below allowed the model to be run successfully on audio data despite any warnings that python may generate during the output.

In [2]:
from datetime import timedelta
import librosa
import numpy as np
import os
import pandas as pd
from scipy.special import expit
import tensorflow as tf
import time
from tqdm import tqdm_notebook

tensorflow: 2.16.1


### Google Colab Steps

If running the notebook in Colab then you'll need to mount Google Drive and clone the repository to it in order to call the models and your recording files when processing.  Skip the next two steps if running on a local machine.

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### Clone the Github repository

If running the notebook through Colab then clone the Github repository to your Google Drive in order to access the model files stored in the repository.

In [ ]:
!git clone https://github.com/ABbiodiversity/Recognizer_Ovenbird.git

#Adjust this link

## Change your working directory
Change your working directory to the folder containing the notebook and the model files.

If running locally then this will be the cloned repository or the folder where you downloaded the files if you didn't clone them with git.

If running in Colab then the location should be the location that the above step cloned into.

This cell will print the working directory so you can double check that it is pointing to the right directory.

In [3]:
# Change the working directory using the %cd command
%cd "C:/Users/Kevin Kelly/Documents/Repos/Recognizer_CommonNighthawk"

C:\Users\Kevin Kelly\Documents\Repos\Recognizer-Common_Nighthawk


C:\Users\Kevin Kelly\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Designate Recording Folder

Enter the full filepath of the folder where the recording files to be processed are stored. The filepath will need to be changed to wherever you have stored the files on your machine or on Google Drive, depending on your processing choice.

Example filepath for local files: 
```python
"C:/Users/User/Documents/Ovenbird_recordings/"
```
If you're unfamiliar with the syntax for Google Drive paths in Colab, "/content/drive/" is required, followed by the rest of the filepath in your Google Drive folder structure, excluding any spaces in the folder names. Example filepath for Google Drive folder: 
```python
"/content/drive/MyDrive/Ovenbird/Recordings_Ovenbird/'
```

In [7]:
# Change path below to match your recordings folder filepath
RECORDINGS = 'C:/Users/Kevin Kelly/Documents/Repos/Recognizer_CommonNighthawk/Recordings-test/' # Change Me to match your recordings filepath

### List the recording files
If you want to check that you have correctly designated the folderpath of your target recordings and that the folder contains the recording files then you can run the following cell to list all the contents of your RECORDINGS folder.

In [8]:
os.listdir(RECORDINGS)

['01_s_EDR-04-0000_20160713_220000.wav',
 'Matchedbyuser_20230523_184925.wav',
 'Matchedbyuser_20230523_184925_detection_function-64-1.csv',
 'Matchedbyuser_20230523_184925_detection_function-64-32.csv',
 'Matchedbyuser_20230523_184925_detection_function-64-64.csv',
 'results']

# Run the Common Nighthawk recognizer on your recordings

The following script will run the audio files in your "RECORDINGS" folder through the Ovenbird recognizer and export the results for each recording file.  

<b> Redo tensorflow model file names and paths </b>

In [9]:
def spectrogram_from_audio(y, sr):
    sr = 16000
    n_fft = 64
    hop_length = 32
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=np.blackman)[1:,:] # remove DC
    D = librosa.power_to_db(np.abs(D)**2, ref=1.0)
    D = np.swapaxes(D,0,1)[np.newaxis,np.newaxis,:,:] # [batch,height,width,channels]
    return D

def process_directory(recordings):
    # make sure there is somewhere to save results
    results_location = os.path.join(recordings, 'results')
    if not os.path.exists(results_location):
        os.makedirs(results_location)
    
    # find tensorflow model files
    meta_graph_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'coni-model.meta')
    checkpoint_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'coni-model')
    
    # list all audio files in directory
    audio_files = librosa.util.find_files(recordings, ext=['flac', 'wav'], recurse=True, case_sensitive=False, limit=None, offset=0)
    audio_files = list(set(audio_files)) # remove duplicates - not sure why find_files produces them!
    
    tf.compat.v1.reset_default_graph()
    with tf.compat.v1.Session() as sess:
        new_saver = tf.compat.v1.train.import_meta_graph(meta_graph_path)
        new_saver.restore(sess, checkpoint_path)
        features = tf.compat.v1.get_collection('input')[0]
        is_training = tf.compat.v1.get_collection('is_training')[0]
        level = tf.compat.v1.get_collection('level')[0]
        detection_fn = tf.compat.v1.get_collection('detection_fn')[0]
        
        for filename in tqdm_notebook(audio_files):
            y, sr = librosa.load(filename, sr=16000, mono=True)
            S = spectrogram_from_audio(y, sr)
            level_dB, detection_function = sess.run([level, detection_fn], feed_dict={features: S, is_training: False})
            level_dB = level_dB[0,0,:,0]
            detection_function = detection_function[0,0,:,0]
            duration_secs = y.size / float(sr) 
            times = np.interp(np.arange(detection_function.size), [0,detection_function.size-1], [0,duration_secs])
            df = pd.DataFrame({'detection_window_start_time': times, 'detector_score': detection_function, 'relative_sound_level': level_dB})
        
            base = os.path.basename(filename)
            filename = os.path.splitext(base)[0]
            savename = os.path.join(results_location, filename + '_detection_function.csv')
            df.to_csv(savename, header=True, index=False)

process_directory(RECORDINGS)

'local_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from C:\Users\Kevin Kelly\Documents\Repos\Recognizer-Common_Nighthawk\coni-model


C:\Users\Kevin Kelly\AppData\Local\Temp\ipykernel_18788\373066670.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for filename in tqdm_notebook(audio_files):


  0%|          | 0/2 [00:00<?, ?it/s]

# Results

Results files will be exported in csv format to a folder called "results" within the RECORDINGS folder.  The files will have the same name as the audio recordings files with "_detection_function" as a suffix.  Each row in the files will consist of the detection window start time, the recognizer score (between 0 and 1) for that detection window, and the relative sound level for that detection window.